## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028B300BDDE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028B300778B0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish! It's nice to meet you.  \n\nAs a Chief AI Engineer, I imagine you have a lot on your plate. What kind of projects are you currently working on? \n\nI'm always interested in learning about the latest developments in the field of AI.  Perhaps you could tell me about some of the challenges and opportunities you see in your work?\n", response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 22, 'total_tokens': 102, 'completion_time': 0.145454545, 'prompt_time': 0.000155849, 'queue_time': 0.013190101000000001, 'total_time': 0.145610394}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3481a2be-80d0-4056-a5cb-99dd6896c7bb-0', usage_metadata={'input_tokens': 22, 'output_tokens': 80, 'total_tokens': 102})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Krish, and that you are a Chief AI Engineer! 😊 \n\nIs there anything else you'd like to tell me about your work?  I'm curious to learn more. \n", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 97, 'total_tokens': 145, 'completion_time': 0.087272727, 'prompt_time': 0.002671011, 'queue_time': 0.011358429, 'total_time': 0.089943738}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2fab9745-8554-45fd-a6be-352894ffdce3-0', usage_metadata={'input_tokens': 97, 'output_tokens': 48, 'total_tokens': 145})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [9]:
response.content

"Hello Krish, it's nice to meet you! \n\nThat's a fantastic title. As a Chief AI Engineer, I imagine you're at the forefront of some exciting projects. \n\nWhat kind of work are you currently focused on?  \n\nI'm always eager to learn more about the cutting-edge applications of AI.\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  I remember you told me at the beginning of our conversation! 😊  \n\nIs there anything else I can help you with today, Krish? \n', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 109, 'total_tokens': 147, 'completion_time': 0.069090909, 'prompt_time': 0.006683756, 'queue_time': 0.006681683, 'total_time': 0.075774665}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-168868a6-819a-4108-b092-f75e36bf49f5-0', usage_metadata={'input_tokens': 109, 'output_tokens': 38, 'total_tokens': 147})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to remember it! 😊  \n\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. \n\nWhat can I do for you today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John,  I remember! 😊  \n\nIs there anything else I can help you with?'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 30, 'total_tokens': 57, 'completion_time': 0.049090909, 'prompt_time': 0.00013335, 'queue_time': 0.014437199999999999, 'total_time': 0.049224259}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aecb1ad1-0cf3-4611-a50c-2271cc234b6f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 30, 'total_tokens': 56, 'completion_time': 0.047272727, 'prompt_time': 0.000176879, 'queue_time': 0.013417001, 'total_time': 0.047449606}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e75198cf-d15d-4857-a884-f9bf2baffc75-0', usage_metadata={'input_tokens': 30, 'output_tokens': 26, 'total_tokens': 56})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  I remember.  😊  How can I help you further? \n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्णा! \n\nमुझे बहुत खुशी है कि आपने मुझे अपना नाम बताया। \n\nमैं आपकी सहायता करने के लिए यहाँ हूँ। \n\nआप मुझसे क्या पूछना चाहते हैं? \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण!  मुझे खुशी है तुम्हारी मुलाक़ात करने में। \n\nआप कैसे हैं? में आपकी मदद कैसे कर सकता हूँ ?\n'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [24]:
response.content

'आपका नाम कृष्ण है, मैंने पहले ही यह जान लिया था! 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"